# Homework 2

## Exercise 3

Let $A\in\mathbb M_{nxn}$ with spectral radius $r(A)<1$.
Take an arbitrary $\epsilon$ satisfying $0<\epsilon<1-r(A)$. 
By Gelfand's formula we know that there exists a $K(\epsilon)\in\mathbb N$ such that for any $k\geq K(\epsilon)$ we have 
$$(r(A)-\epsilon)^k<||A^k||<(r(A)+\epsilon)^k.$$

Since $||A^k||\geq 0$, we can rewrite the previous inequalities as
$$0<||A^k||<(r(A)+\epsilon)^k.$$
By our choice of $\epsilon$ we know that 
$(r(A)+\epsilon)^k\to 0$ as $k\to\infty$.
Therefore $||A^k||\to 0$ as $k\to\infty$.